In [3]:
!pip install bs4
!pip install lxml

  Using cached https://files.pythonhosted.org/packages/10/ed/7e8b97591f6f456174139ec089c769f89a94a1a4025fe967691de971f314/bs4-0.0.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/d1/41/e6495bd7d3781cee623ce23ea6ac73282a373088fcd0ddc809a047b18eae/beautifulsoup4-4.9.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/fb/1c65691a9aeb7bd6ac2aa505b84cb8b49ac29c976411c6ab3659425e045f/soupsieve-2.1-py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 5.5MB 4.9MB/s eta 0:00:01�███               | 2.9MB 4.9MB/s eta 0:00:01


In [4]:
import pandas as pd
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
from lxml import etree
import numpy as np

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page=requests.get(url)
print(page.status_code) #If the response is 200, we can go ahead for scraping
data = pd.read_html(url) #Read the table into dataframe

200


In [6]:
toronto_df = data[0]

#Ignore cells with a borough that is Not assigned.
BoroughNotAssigned = toronto_df[toronto_df['Borough'] == 'Not assigned'].index
toronto_df.drop(BoroughNotAssigned,inplace=True)

#If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough
toronto_df['Neighbourhood'] = np.where(toronto_df['Neighbourhood'] == 'Not assigned', toronto_df['Borough'], toronto_df['Neighbourhood'])

toronto_df 

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
toronto_df.shape

(103, 3)

In [8]:
#Import Geographical coordinates of each postal code 
!wget -q -O 'geocoordinates_postalcode.csv' http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
print('Data downloaded!')

Data downloaded!


In [33]:
geocoord_df = pd.read_csv('geocoordinates_postalcode.csv')
geocoord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
# Inner join will be used to merge the two dataframe.
# Each row that have similar Postal Code will be merged to give new dataframe = coplete_df
complete_df = toronto_df.merge(geocoord_df)

In [36]:
complete_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
